# Regularization

Let's improve our understanding of what impacted **Titanic** passengers' chance of survival
- We will use logistic classifiers which are easy to interpret
- Remember we already did it with statsmodels in lecture "Decision Science - Logistic Regression"
- We were using `p-values` & statistical assumptions to detect which features were irrelevant / don't generalize
- This time, we will use `regularization` to detect relevant/irrelevant features based on under/overfitting criteria
- **Our goal is to compare `L1` and `L2` penalties**

## 1. We load and preprocess the data for you

In [36]:
import pandas as pd
import numpy as np

In [37]:
data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/ML_titanic_dataset_encoded.csv")

# the dataset is already one-hot-encoded
data.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [38]:
# We build X and y

y = data["survived"]
X = data.drop(columns="survived")
X.head()

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [39]:
# We MinMaxScale our features for you
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X.shape

(714, 12)

## 1.  Logistic Regression without regularization

❓ Rank the features by decreasing order of importance after training a simple **non-regularized** Logistic Regression (i.e. look at the coefficients after fitting)
- Careful: `LogisticRegression` is penalized by default
  - take a look at the [penalty parameter](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) to find out how to remove the penalty)
- Increase `max_iter` to a larger number until model converges

<details>
    <summary>Hint</summary>
    <img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/05-ML/05-Model-Tuning/model_selection.png" alt="penalizing a regression" width="500" height="600">
</details>

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
LR = LogisticRegression(penalty=None,  max_iter=1000)

LR.fit(X_train, y_train)

coef = LR.coef_[0]
features = X_train.columns

coef_df = pd.DataFrame({
    'feature': features,
    'coef': coef,
    'abs_coef': np.abs(coef)  # para ordenar por importância
})
coef_df = coef_df.sort_values(by='abs_coef', ascending=False) 
coef_df

,feature,coef,abs_coef
5,sex_female,2.858771,2.858771
8,who_child,1.520305,1.520305
6,class_First,1.070863,1.070863
7,class_Third,-0.677114,0.677114
2,sibsp,-0.491403,0.491403
0,pclass,-0.482164,0.482164
9,embark_town_Cherbourg,0.375837,0.375837
3,parch,-0.192465,0.192465
10,embark_town_Queenstown,0.143489,0.143489
11,embark_town_Southampton,0.035495,0.035495


❓How do you interpret, in plain English, the value for the coefficient `sex_female`?

<details>
    <summary>Answer</summary>

> "All other things being equal (such as age, ticket class etc...),
being a women increases your log-odds of survival by 2.67 (your coef value)"
    
> "Controling for all other explaining factors available in this dataset,
being a women increases your odds of survival by exp(2.67) = 14"

</details>


❓ What is the feature that most impacts the chances of survival according to your model?  
Fill the `top_1_feature` list below with the name of this feature

In [49]:
top_1_feature = ['embark_town_Cherbourg']

In [50]:
from nbresult import ChallengeResult
result = ChallengeResult('unregularized', top_1_feature = top_1_feature)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/andreza/.pyenv/versions/3.10.6/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /home/andreza/code/andrezaumbelino/05-ML/05-Model-Tuning/data-regularization/tests
plugins: typeguard-2.13.3, asyncio-0.19.0, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_unregularized.py::TestUnregularized::test_top_1 PASSED              [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/unregularized.pickle

git commit -m 'Completed unregularized step'

git push origin master



## 2.  Logistic Regression with an L2 penalty

Let's use a **Logistic model** whose log-loss has been penalized with a **L2** term to figure out the **most important features** without overfitting.  
This is the "classification" equivalent to the "Ridge" regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its features by importance (look at the coefficients)
- By "strongly regularized" we mean "more than Sklearn's default regularization factor". 
- Sklearn's default values are very useful orders of magnitudes to keep in mind for "scaled features"

In [52]:
LR2 = LogisticRegression(penalty='l2',  max_iter=1000)

LR2.fit(X_train, y_train)

coef2 = LR2.coef_[0]
features2 = X_train.columns

coef_df2 = pd.DataFrame({
    'feature2': features2,
    'coef2': coef2,
    'abs_coef2': np.abs(coef2)  # para ordenar por importância
})
coef_df2 = coef_df2.sort_values(by='abs_coef2', ascending=False) 
coef_df2

,feature2,coef2,abs_coef2
5,sex_female,2.630094,2.630094
8,who_child,1.123404,1.123404
0,pclass,-0.854434,0.854434
6,class_First,0.586840,0.586840
2,sibsp,-0.418107,0.418107
7,class_Third,-0.282943,0.282943
11,embark_town_Southampton,-0.190423,0.190423
9,embark_town_Cherbourg,0.157120,0.157120
3,parch,-0.154113,0.154113
10,embark_town_Queenstown,-0.055329,0.055329


❓ What are the top 2 features driving chances of survival according to your model?  
Fill the `top_2_features` list below with the name of these features

In [53]:
top_2_features = ["sex_female", "who_child"]

#### 🧪 Test your code below

In [54]:
from nbresult import ChallengeResult
result = ChallengeResult('ridge', top_2 = top_2_features)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/andreza/.pyenv/versions/3.10.6/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /home/andreza/code/andrezaumbelino/05-ML/05-Model-Tuning/data-regularization/tests
plugins: typeguard-2.13.3, asyncio-0.19.0, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_ridge.py::TestRidge::test_top2 PASSED                               [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/ridge.pickle

git commit -m 'Completed ridge step'

git push origin master



## 2. Logistic Regression with an L1 penalty

This time, we'll use a logistic model whose log-loss has been penalized with a **L1** term to **filter-out the less important features**.  
This is the "classification" equivalent to the **Lasso** regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its features by importance

In [60]:
LR3 = LogisticRegression(C=1, penalty='l1', solver='liblinear')

LR3.fit(X_train, y_train)

coef3 = LR3.coef_[0]
features3 = X_train.columns

coef_df3 = pd.DataFrame({
    'feature3': features3,
    'coef3': coef3,
    'abs_coef3': np.abs(coef3)  # para ordenar por importância
})
coef_df3 = coef_df3.sort_values(by='abs_coef3', ascending=False) 
coef_df3

,feature3,coef3,abs_coef3
5,sex_female,2.736401,2.736401
6,class_First,1.264912,1.264912
8,who_child,1.240049,1.240049
7,class_Third,-0.880516,0.880516
2,sibsp,-0.421173,0.421173
9,embark_town_Cherbourg,0.269076,0.269076
0,pclass,-0.170382,0.170382
3,parch,-0.168512,0.168512
1,age,-0.031765,0.031765
4,fare,-0.000144,0.000144


❓ What are the features that have absolutely no impact on chances of survival, according to your L1 model?  
Fill the `zero_impact_features` list below with the name of these features; you may have to add elements to the list.

- Do you notice how some of them were "highly important" according to the non-regularized model? 
- From now on, we will always regularize our linear models!

In [61]:
zero_impact_features = ["embark_town_Queenstown", "embark_town_Southampton", "fare", "age"]

#### 🧪 Test your code below

In [62]:
from nbresult import ChallengeResult
result = ChallengeResult('lasso', zero_impact_features = zero_impact_features)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/andreza/.pyenv/versions/3.10.6/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /home/andreza/code/andrezaumbelino/05-ML/05-Model-Tuning/data-regularization/tests
plugins: typeguard-2.13.3, asyncio-0.19.0, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_lasso.py::TestLasso::test_zero_impact PASSED                        [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/lasso.pickle

git commit -m 'Completed lasso step'

git push origin master



**🏁 Congratulation! Don't forget to commit and push your notebook**